# Phonopy workflow ideas

In [1]:
%%time
%config IPCompleter.evaluation='unsafe'

import matplotlib.pylab as plt
from pyiron_workflow.workflow import Workflow
from pyiron_workflow.function import single_value_node

CPU times: user 1.35 s, sys: 251 ms, total: 1.6 s
Wall time: 326 ms


In [2]:
%%time
wf = Workflow('phonopy')
wf.register('structure', 'pyiron_workflow.node_library.structure')
wf.register('calculator', 'pyiron_workflow.node_library.calculator')
wf.register('engine', 'pyiron_workflow.node_library.engine')
wf.register('phonopy', 'pyiron_workflow.node_library.phonopy')

CPU times: user 25.5 ms, sys: 7.3 ms, total: 32.8 ms
Wall time: 34.4 ms


In [3]:
from pyiron_workflow.node_library.phonopy import InputPhonopyGenerateSupercells

In [4]:
@Workflow.wrap_as.macro_node(
    "imaginary_modes", "total_dos", "energy_relaxed", "energy_initial", "energy_displaced"
)
def run_phonopy(wf, element: str, cell_size: int = 2, vacancy_index: int | None = None, displacement: float = 0.01):
    wf.structure = wf.create.structure.build.cubic_bulk_cell(
        element=element, cell_size=cell_size, vacancy_index=vacancy_index
    )
    # explicit output needed since macro and not single_value_node (we should have also a single_value_macro)
    wf.relaxed_structure = wf.create.calculator.ase.minimize(
        atoms=wf.structure.outputs.structure
    )

    wf.engine = wf.create.engine.ase.M3GNet()

    wf.phonopy = wf.create.phonopy.create_phonopy(
        structure=wf.relaxed_structure.outputs.structure,
        parameters=InputPhonopyGenerateSupercells(distance=displacement.run()),
        engine=wf.engine
    )
    # print ('test: ', displacement.run())

    wf.check_consistency = wf.create.phonopy.check_consistency(phonopy=wf.phonopy.outputs.phonopy)
    wf.total_dos = wf.create.phonopy.get_total_dos(phonopy=wf.phonopy.outputs.phonopy)

    # iterate over all nodes, extract the log_output and store it in hdf5
    # control the amount of output via log_level

    return (
        wf.check_consistency,
        wf.total_dos,
        wf.relaxed_structure.outputs.out["energy"],
        wf.relaxed_structure.outputs.out["energy_initial"],
        wf.phonopy.outputs.out["energies"]
    )

In [5]:
wf = run_phonopy(element='Al', cell_size=3, vacancy_index=0, displacement=0.1)
wf.run()

energy:  0.8013167095855369 max_force:  2.2551405187698492e-17


/Users/joerg/mambaforge/envs/intel11/lib/python3.11/site-packages/matgl/apps/pes.py:59: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.element_refs = AtomRef(property_offset=torch.tensor(element_refs, dtype=matgl.float_th))
/Users/joerg/mambaforge/envs/intel11/lib/python3.11/site-packages/matgl/apps/pes.py:65: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  self.register_buffer("data_mean", torch.tensor(data_mean, dtype=matgl.float_th))
/Users/joerg/mambaforge/envs/intel11/lib/python3.11/site-packages/matgl/apps/pes.py:66: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather th

alles ok


{'imaginary_modes': False,
 'total_dos':      frequency_points  total_dos
 0           -0.430914        0.0
 1           -0.394666        0.0
 2           -0.358418        0.0
 3           -0.322169        0.0
 4           -0.285921        0.0
 ..                ...        ...
 196          6.673746        0.0
 197          6.709995        0.0
 198          6.746243        0.0
 199          6.782491        0.0
 200          6.818739        0.0
 
 [201 rows x 2 columns],
 'energy_relaxed': 0.8013167095855369,
 'energy_initial': 0.7996059979144441,
 'energy_displaced': 0     -395.84808
 1      -395.8479
 2     -395.84735
 3     -395.84863
 4     -395.84808
 5     -395.84796
 6      -395.8479
 7      -395.8482
 8     -395.84763
 9     -395.84842
 10    -395.84793
 11    -395.84814
 12    -395.84802
 13    -395.84802
 14    -395.84802
 15    -395.84802
 16    -395.84802
 17    -395.84808
 18    -395.84796
 19    -395.84802
 20    -395.84802
 Name: energy, dtype: object}

In [6]:
wf.phonopy.outputs.out.value['df']

,atoms,_internal,forces,energy,iter_index
0,"(Atom('Al', [12.157071067811865, 2.01846947349...",{'iter_index': [0]},"[[-0.017570268, -0.00016299635, 0.016304713], ...",-395.84808,[0]
1,"(Atom('Al', [12.14292893218813, 2.004327337874...",{'iter_index': [1]},"[[0.01760599, 0.02923628, 0.012760781], [0.014...",-395.8479,[1]
2,"(Atom('Al', [12.149999999999997, 2.01139840568...",{'iter_index': [2]},"[[-0.00043985137, 0.012513507, 0.016508885], [...",-395.84735,[2]
3,"(Atom('Al', [12.149999999999997, 2.01139840568...",{'iter_index': [3]},"[[0.00043296517, 0.016630674, 0.012541054], [0...",-395.84863,[3]
4,"(Atom('Al', [12.149999999999997, 2.01139840568...",{'iter_index': [4]},"[[-0.00013847595, 0.014457194, 0.014981069], [...",-395.84808,[4]
5,"(Atom('Al', [12.149999999999997, 2.01139840568...",{'iter_index': [5]},"[[0.00011192348, 0.014630016, 0.014106842], [0...",-395.84796,[5]
6,"(Atom('Al', [12.149999999999997, 2.01139840568...",{'iter_index': [6]},"[[0.002472178, 0.014731701, 0.017395992], [0.0...",-395.8479,[6]
7,"(Atom('Al', [12.149999999999997, 2.01139840568...",{'iter_index': [7]},"[[-0.0025749928, 0.014353024, 0.011581592], [0...",-395.8482,[7]
8,"(Atom('Al', [12.149999999999997, 2.01139840568...",{'iter_index': [8]},"[[0.0028479605, 0.014593927, 0.017195765], [0....",-395.84763,[8]
9,"(Atom('Al', [12.149999999999997, 2.01139840568...",{'iter_index': [9]},"[[-0.002984147, 0.014497297, 0.011809291], [0....",-395.84842,[9]


In [7]:
xx

NameError: name 'xx' is not defined

### Pseudocode for output class

In [ ]:
from typing import Optional, Union
from typing import Callable, TypeVar, Any, TypeAlias
from dataclasses import dataclass

import numpy as np

In [ ]:
@dataclass
class VarFunc:
    func: Callable = None
    log_level: int = 0
    unit: str = ''

In [ ]:
@dataclass
class VarType:
    property: TypeVar = None
    log_level: int = 0
    unit: str = ''

In [ ]:
VarFunc(func=np.sin, log_level=2).func

In [ ]:
class toy_job:
    def __init__(self, x=0):
        self.x = x

    def get_energy(self):
        return np.sin(self.x)

    def get_forces(self):
        return np.ones(3)

    @property
    def my_x(self):
        return self.x

In [ ]:
job = toy_job(1)
job.get_energy(), job.get_forces()

In [ ]:
@dataclass
class wfOutput:
    pass

    def keys(self):
        return self.__dict__.keys()

    def __getitem__(self, key):
        return self.__dict__[key]    

    def __call__(self, job):
        out_dict = dict()
        for key in self.keys():
            print (key)
            v = self[key]
            if isinstance(v, VarFunc):
                out_dict[key] = job.__getattribute(job)()
                
        return out_dict       

In [ ]:
import pint
ureg = pint.UnitRegistry()
ureg.angstrom

In [ ]:
# import ase
import pint

@dataclass
class OutputEnergyStatic:
    distance: float = 0.01
    energy: VarFunc = VarFunc(func=toy_job.get_energy, log_level=0, unit=ureg.eV)
    forces: VarFunc = VarFunc(func=toy_job.get_forces, log_level=1, unit=ureg.eV/ureg.angstrom)
    prop: VarType = VarType(toy_job.my_x, log_level=2)

    def keys(self):
        return self.__dict__.keys()

    def __getitem__(self, key):
        return self.__dict__[key]    

    def __call__(self, job):
        out_dict = dict()
        for key in self.keys():
            print (key)
            v = self[key]
            if isinstance(v, VarFunc):
                out_dict[key] = job.__getattribute(job)()
                
        return out_dict 

In [ ]:
output = OutputEnergyStatic()
output.__dict__

In [ ]:
def calc_static(atoms, output=OutputEnergyStatic()):
    atoms.calc = EMT()


    output['energy'] = atoms.get_energy()
    
    return output(atoms)

In [ ]:
def calc_static(atoms, output=wfOutput()):
    atoms.calc = EMT()

    output = dict()
    output['energy'] = atoms.get_energy()
    
    return output

In [ ]:
def calc_static(atoms):
    atoms.calc = EMT()

    output = wfOutput()
    output['energy'] = atoms.get_energy()
    
    return output

In [ ]:
wf.output = wf.create.calc_static.output(keys='energy')
wf.calc = wf.create.calc_static(output=wf.output)

In [ ]:
xx

In [ ]:
%%time
df = wf.iter(cell_size=list(range(1,4)), element=['Al'], vacancy_index=[None, 0], displacement=[0.01, 0.1]) #, Cu, Pd, Ag, Pt and Au])

In [ ]:
df

In [ ]:
df.energy_displaced

In [ ]:
xx

In [ ]:
@single_value_node()
def calc_static(atoms=None, engine=None): 
    print ('atoms: ', atoms)
    if engine is None:
        from ase.calculators.emt import EMT
        calculator = EMT() 

    atoms.calc = calculator

    out = {}
    # out['structure'] = atoms # not needed since identical to input
    out['forces'] = atoms.get_forces()
    out['energy'] = atoms.get_potential_energy()
  
    return out    

In [ ]:
#@single_value_node()
def generate_supercells(phonopy):
    from structuretoolkit.common import phonopy_to_atoms
    
    phonopy.generate_displacements()  

    supercells = [phonopy_to_atoms(s) for s in phonopy.supercells_with_displacements]
    return supercells

In [ ]:
@single_value_node()
def iter(node, qwargs):
    out = node.iter(**qwargs)
    return out

wf = Workflow('phonopy')
wf.structure = wf.create.pyiron_atomistics.Bulk('Al', cubic=True)
wf.supercell = wf.create.lammps.Repeat(structure = wf.structure, repeat_scalar=2)
wf.supercells = generate_supercells(structure=wf.supercell)
wf.calc = calc_static() 

*TODO: allowing to use nodes as input arguments in workflows is crucial but not (yet) available*
wf.df = iter(node=wf.calc, qwargs={'atoms': wf.supercells}) 

In [ ]:
@single_value_node()
def create_vacancy(structure, index: int|None = 0):
    structure = structure.copy()
    if index is not None:
        del structure[index]
    
    return structure   

In [ ]:
@Workflow.wrap_as.macro_node("structure")
def CubicBulkCell(wf, element: str, cell_size: int = 1, vacancy_index: int|None = None):
    wf.structure = wf.create.pyiron_atomistics.Bulk(
        name=element, cubic=True
    )
    wf.cell = wf.create.lammps.Repeat(structure = wf.structure, repeat_scalar=cell_size)

    wf.cell_with_vacancies = create_vacancy(structure=wf.cell, index=vacancy_index)
    return wf.cell_with_vacancies #.outputs.structure

In [ ]:
# The following function should be defined as a workflow macro (presently not possible)

@single_value_node()
def create_phonopy(structure, calculator=None, executor=None):
    from phonopy import Phonopy
    from structuretoolkit.common import atoms_to_phonopy
    
    phonopy = Phonopy(unitcell=atoms_to_phonopy(structure))  
    
    cells = generate_supercells(phonopy)# .run()
    gs = calc_static() 
    df = gs.iter(atoms=cells, executor=executor)
    phonopy.forces = df.forces
    
    return phonopy

In [ ]:
import numpy as np

structure = CubicBulkCell(element='Al', cell_size=2, vacancy_index=0).run().structure
phonopy = create_phonopy(structure).run()


In [ ]:
# @single_value_node()
# def create_phonopy(structure, calculator=None):
#     from phonopy import Phonopy
#     # from phonopy.structure.atoms import PhonopyAtoms
#     from structuretoolkit.common import phonopy_to_atoms, atoms_to_phonopy

#     if calculator is None:
#         from ase.calculators.emt import EMT
#         calculator = EMT()
    
#     phonopy = Phonopy(unitcell=atoms_to_phonopy(structure))  
#     phonopy.generate_displacements()
#     structures, forces, energies = [], [], []

#     for s in phonopy.supercells_with_displacements:
#         atoms = phonopy_to_atoms(s)
#         atoms.calc = calculator
#         structures.append(atoms)
#         forces.append(atoms.get_forces())
#         energies.append(atoms.get_potential_energy())
#     phonopy.forces = forces
    
#     return phonopy

In [ ]:
@single_value_node()
def get_dynamical_matrix(phonopy, q=[0,0,0]):
    import numpy as np
    
    if phonopy.dynamical_matrix is None:
        phonopy.produce_force_constants()
        phonopy.dynamical_matrix.run(q=q)
    dynamical_matrix = np.real_if_close(phonopy.dynamical_matrix.dynamical_matrix)
    # print (dynamical_matrix)
    return dynamical_matrix

In [ ]:
@single_value_node()
def get_eigenvalues(matrix):
    import numpy as np
    
    ew = np.linalg.eigvalsh(matrix)
    return ew

In [ ]:
@single_value_node()
def check_consistency(phonopy, tolerance=1e-10):
    dyn_matrix = get_dynamical_matrix(phonopy).run()
    ew = get_eigenvalues(dyn_matrix).run()
    # print ('ew: ', ew)
    ew_lt_zero = ew[ew < -tolerance]
    if len(ew_lt_zero) > 0:
        print (f'WARNING: {len(ew_lt_zero)} imaginary modes exist')
        has_imaginary_modes = True
    has_imaginary_modes = False 
    print ('alles ok')
    return has_imaginary_modes        

In [ ]:
@single_value_node()
def get_total_dos(phonopy, mesh=3*[10]):
    from pandas import DataFrame
    
    phonopy.produce_force_constants()
    phonopy.run_mesh(mesh=mesh)
    phonopy.run_total_dos()
    total_dos = DataFrame(phonopy.get_total_dos_dict())
    return total_dos

In [ ]:
wf = Workflow('phonopy')

wf.structure = CubicBulkCell(element='Al', cell_size=2, vacancy_index=None)
wf.phonopy = create_phonopy(structure=wf.structure.outputs.structure)  # explicit output needed since macro and not single_value_node (we should have also a sinle_value_macro)
#wf.dynamical_matrix = get_dynamical_matrix(phonopy=wf.phonopy)
wf.check_consistency = check_consistency(phonopy=wf.phonopy)
wf.total_dos = get_total_dos(phonopy=wf.phonopy)

wf.inputs_map = {'structure__cell_size': 'cell_size', 'structure__element': 'element'}
wf.outputs_map = {'total_dos__total_dos': 'total_dos', 
                  'check_consistency__has_imaginary_modes': 'imaginary_modes'
                 }

In [ ]:
wf.draw();

In [ ]:
%%time
df = wf.iter(cell_size=list(range(1,3)), element=['Al', 'Ni']) #, Cu, Pd, Ag, Pt and Au])

In [ ]:
df

In [ ]:
xx

In [ ]:
import matplotlib.pylab as plt

for n in range(2, 5):
    out = wf(supercell__repeat_scalar=n)
    df = out.total_dos #__total_dos
    n_atoms = len(wf.structure.outputs.structure.value)
    plt.plot(df.frequency_points, df.total_dos.values/n**3, label=(4*n**3, n_atoms))
plt.title('Phonon DOS')
plt.legend();

In [ ]:
%%time
import numpy as np
from pympipool import Executor



def calc(i, j, k):
    from mpi4py import MPI
    import time
    
    time.sleep(1)
    size = MPI.COMM_WORLD.Get_size()
    rank = MPI.COMM_WORLD.Get_rank()
    return np.array([i+rank, j, k]), size, rank

cores = 2
with Executor(cores_per_worker=cores, max_workers=1) as p:
    for i in range(0, 8, cores):
        fs = p.submit(calc, i, j=55, k=10)
        print(fs.result())

In [ ]:
%%time
from pympipool import Executor

def iter(func, max_workers=1, cores_per_worker=1, **kwargs):
    key = list(kwargs.keys())[0]
    val = kwargs[key]
    
    with Executor(cores_per_worker=cores_per_worker, max_workers=max_workers) as p:
        iter_dict = {'kwargs': kwargs}
        for i, n in enumerate(val):
            fs = p.submit(wf, **{key: n})
            iter_dict[i] = fs.result()
    return iter_dict        

out = iter(wf, supercell__repeat_scalar=list(range(1,3)))

In [ ]:
%%time
from pympipool import Executor

def iter(func, max_workers=1, cores_per_worker=1, **kwargs):
    # Get the keys and lists from kwargs
    keys = list(kwargs.keys())
    lists = list(kwargs.values())

    # Get the number of dimensions
    num_dimensions = len(keys)

    # Get the length of each list
    lengths = [len(lst) for lst in lists]

    # Initialize indices
    indices = [0] * num_dimensions

    with Executor(cores_per_worker=cores_per_worker, max_workers=max_workers) as p:
        #iter_dict = {'kwargs': kwargs}
        iter_dict = {}
        
        # Create an empty DataFrame to store the results
        df_result = pd.DataFrame(columns=keys)
    
        # Perform multidimensional for loop
        count = 0
        while indices[0] < lengths[0]:
            # Access the current elements using indices
            current_elements = [lists[i][indices[i]] for i in range(num_dimensions)]
    
            # Add current_elements as a dictionary
            current_elements_kwarg = dict(zip(keys, current_elements))
            fs = p.submit(wf, **current_elements_kwarg)
            out = fs.result()
            iter_dict[count] = out
            count += 1

            for k, v in out.items():
                current_elements_kwarg[k] = v
                
            # Append the current_elements_kwarg to the DataFrame
            df_result = pd.concat([df_result, pd.DataFrame([current_elements_kwarg])], ignore_index=True)
    
            # Process the current elements (you can modify this part according to your needs)
            #print(f"Current Elements: {current_elements_kwarg}")
    
            # Update indices for the next iteration
            indices[num_dimensions - 1] += 1
    
            # Update indices and carry-over if needed
            for i in range(num_dimensions - 1, 0, -1):
                if indices[i] == lengths[i]:
                    indices[i] = 0
                    indices[i - 1] += 1
    return df_result        
       
df = iter(wf, cell_size=list(range(1,3)))

In [ ]:
%%time
df.to_hdf('test.h5', 'table')

In [ ]:
df.total_dos.values[1].plot();

In [ ]:
out[1]['total_dos'].plot(x='frequency_points');

In [ ]:
def multidimensional_for_loop(**kwargs):
    # Get the keys and lists from kwargs
    keys = list(kwargs.keys())
    lists = list(kwargs.values())

    # Get the number of dimensions
    num_dimensions = len(keys)

    # Get the length of each list
    lengths = [len(lst) for lst in lists]

    # Initialize indices
    indices = [0] * num_dimensions

    # Perform multidimensional for loop
    while indices[0] < lengths[0]:
        # Access the current elements using indices
        current_elements = [lists[i][indices[i]] for i in range(num_dimensions)]

        # Process the current elements (you can modify this part according to your needs)
        print(f"Current Elements: {current_elements}")

        # Update indices for the next iteration
        indices[num_dimensions - 1] += 1

        # Update indices and carry-over if needed
        for i in range(num_dimensions - 1, 0, -1):
            if indices[i] == lengths[i]:
                indices[i] = 0
                indices[i - 1] += 1

    # You can add additional processing or return values as needed

# Example usage
list1 = [1, 2]
list2 = ['a', 'b', 'c']
list3 = [True, False]

multidimensional_for_loop(arg1=list1, arg2=list2, arg3=list3)

In [ ]:
def multidimensional_for_loop(**kwargs):
    # Get the keys and lists from kwargs
    keys = list(kwargs.keys())
    lists = list(kwargs.values())

    # Get the number of dimensions
    num_dimensions = len(keys)

    # Get the length of each list
    lengths = [len(lst) for lst in lists]

    # Initialize indices
    indices = [0] * num_dimensions

    # Perform multidimensional for loop
    while indices[0] < lengths[0]:
        # Access the current elements using indices
        current_elements = [lists[i][indices[i]] for i in range(num_dimensions)]

        # Add current_elements as a keyword argument
        current_elements_kwarg = dict(zip(keys, current_elements))
        kwargs.update({'current_elements': current_elements_kwarg})

        # Process the current elements (you can modify this part according to your needs)
        print(f"Current Elements: {current_elements_kwarg}")

        # Update indices for the next iteration
        indices[num_dimensions - 1] += 1

        # Update indices and carry-over if needed
        for i in range(num_dimensions - 1, 0, -1):
            if indices[i] == lengths[i]:
                indices[i] = 0
                indices[i - 1] += 1

    # You can add additional processing or return values as needed

# Example usage
list1 = [1, 2]
list2 = ['a', 'b', 'c']
list3 = [True, False]

In [ ]:
multidimensional_for_loop(a=list1, b=list2, c=list3)

In [ ]:
import pandas as pd

def multidimensional_for_loop(**kwargs):
    # Get the keys and lists from kwargs
    keys = list(kwargs.keys())
    lists = list(kwargs.values())

    # Get the number of dimensions
    num_dimensions = len(keys)

    # Get the length of each list
    lengths = [len(lst) for lst in lists]

    # Initialize indices
    indices = [0] * num_dimensions

    # Create an empty DataFrame to store the results
    df_result = pd.DataFrame(columns=keys)

    # Perform multidimensional for loop
    while indices[0] < lengths[0]:
        # Access the current elements using indices
        current_elements = [lists[i][indices[i]] for i in range(num_dimensions)]

        # Add current_elements as a dictionary
        current_elements_kwarg = dict(zip(keys, current_elements))

        # Append the current_elements_kwarg to the DataFrame
        df_result = pd.concat([df_result, pd.DataFrame([current_elements_kwarg])], ignore_index=True)

        # Process the current elements (you can modify this part according to your needs)
        print(f"Current Elements: {current_elements_kwarg}")

        # Update indices for the next iteration
        indices[num_dimensions - 1] += 1

        # Update indices and carry-over if needed
        for i in range(num_dimensions - 1, 0, -1):
            if indices[i] == lengths[i]:
                indices[i] = 0
                indices[i - 1] += 1

    # You now have a DataFrame containing the results
    print("DataFrame Result:")
    print(df_result)

    # You can add additional processing or return values as needed

# Example usage
list1 = [1, 2]
list2 = ['a', 'b', 'c']
list3 = [True, False]

multidimensional_for_loop(a=list1, b=list2, arg3=list3)

In [ ]:
from dataclasses import dataclass
from typing import Optional, Union

@dataclass
class InputPhonopyGenerateSupercells:
    distance: float = 0.01
    is_plusminus: Union[str, bool] = "auto"
    is_diagonal: bool = True
    is_trigonal: bool = False
    number_of_snapshots: Optional[int] = None
    random_seed: Optional[int] = None
    temperature: Optional[float] = None
    cutoff_frequency: Optional[float] = None
    max_distance: Optional[float] = None

    def keys(self):
        return self.__dict__.keys()
        
    def __getitem__(self, key):
        return self.__dict__[key]

In [ ]:
par = InputPhonopyGenerateSupercells(distance=0.02)

In [ ]:
def f(distance=0.01, **kwargs):
    print (kwargs)
    print (distance)

In [ ]:
f(**par)

In [ ]:
f(**{'distance': 0.03})

In [ ]:
dict_lst = {}
for count in range(1, 4):
    current_elements_kwarg = {'a': 1}
    _internal = {}
    _internal["iter_index"] = count
    current_elements_kwarg["_internal"] = _internal
    print("indices: ", _internal)    

    for k, v in current_elements_kwarg.items():
        if count == 1:
             dict_lst[k] = [v]
        else:    
            if k in dict_lst:
                dict_lst[k].append(v)
            else:
                ValueError(f"New key appears at count {count}")      
    
dict_lst

### Test universal ML potential

In [ ]:
from ase import units
from ase.build import bulk
from atomistics.calculators import calc_molecular_dynamics_langevin_with_ase

In [ ]:
import matgl
from matgl.ext.ase import M3GNetCalculator

In [ ]:
%%time
structure = bulk("Al", cubic=True).repeat([3, 3, 3])
ase_calculator = M3GNetCalculator(matgl.load_model("M3GNet-MP-2021.2.8-PES"))
result_dict = calc_molecular_dynamics_langevin_with_ase(
    structure=structure,
    ase_calculator=ase_calculator,
    run=1000,
    thermo=10,
    timestep=1 * units.fs,
    temperature=1000,
    friction=0.002,
)

In [ ]:
result_dict.keys()

In [ ]:
plt.plot(result_dict['energy_pot']);

In [ ]:
plt.plot(result_dict['positions'][:,:,0]);